## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✅ Libraries imported successfully")

In [ ]:
# Load the merged output data for resource analysis
df = pd.read_csv('../output/output_merged.csv')

# Load output.csv for time series trend analysis (has better time granularity)
df_trends = pd.read_csv('../output/output.csv')

# Display basic information
print(f"📊 Main Dataset Shape: {df.shape[0]:,} rows x {df.shape[1]} columns")
print(f"📊 Trends Dataset Shape: {df_trends.shape[0]:,} rows x {df_trends.shape[1]} columns")
print(f"\n📅 Main Date Range: {df['timeUsageStarted'].min()} to {df['timeUsageEnded'].max()}")
print(f"📅 Trends Date Range: {df_trends['timeUsageStarted'].min()} to {df_trends['timeUsageEnded'].max()}")
print(f"\n💰 Total Cost (Main): ${df['computedAmount'].sum():,.2f}")
print(f"💰 Total Cost (Trends): ${df_trends['computedAmount'].sum():,.2f}")

# Show first few rows
df.head()

## 2. Data Cleaning and Preparation

In [ ]:
# Convert date columns to datetime for main dataset
df['timeUsageStarted'] = pd.to_datetime(df['timeUsageStarted'])
df['timeUsageEnded'] = pd.to_datetime(df['timeUsageEnded'])

# Add date components for time series analysis
df['date'] = df['timeUsageStarted'].dt.date
df['year'] = df['timeUsageStarted'].dt.year
df['month'] = df['timeUsageStarted'].dt.month
df['month_name'] = df['timeUsageStarted'].dt.strftime('%Y-%m')
df['day_of_week'] = df['timeUsageStarted'].dt.day_name()
df['hour'] = df['timeUsageStarted'].dt.hour

# Clean compartment paths - extract meaningful names
df['compartment_name_clean'] = df['compartmentPath'].fillna('Unknown').str.split('/').str[-1]

# Fill missing values
df['service'] = df['service'].fillna('Unknown Service')
df['region_from_call2'] = df['region_from_call2'].fillna(df['region'])
df['shape_from_call2'] = df['shape_from_call2'].fillna('No Shape')

# Process trends dataset for time series analysis
df_trends['timeUsageStarted'] = pd.to_datetime(df_trends['timeUsageStarted'])
df_trends['timeUsageEnded'] = pd.to_datetime(df_trends['timeUsageEnded'])
df_trends['date'] = df_trends['timeUsageStarted'].dt.date
df_trends['hour'] = df_trends['timeUsageStarted'].dt.hour
df_trends['service'] = df_trends['service'].fillna('Unknown Service')
df_trends['compartment_name_clean'] = df_trends['compartmentPath'].fillna('Unknown').str.split('/').str[-1]

print("✅ Main dataset cleaned and prepared")
print(f"📊 Unique Services: {df['service'].nunique()}")
print(f"📊 Unique Regions: {df['region_from_call2'].nunique()}")
print(f"📊 Unique Compartments: {df['compartment_name_clean'].nunique()}")

print("\n✅ Trends dataset prepared for time series analysis")
print(f"📊 Trends - Unique Services: {df_trends['service'].nunique()}")
print(f"📊 Trends - Time Granularity: Hourly data available")

## 3. Executive Summary - Key Metrics

In [ ]:
# Calculate key metrics
total_cost = df['computedAmount'].sum()
daily_avg_cost = df.groupby('date')['computedAmount'].sum().mean()
top_service = df.groupby('service')['computedAmount'].sum().idxmax()
top_service_cost = df.groupby('service')['computedAmount'].sum().max()
top_region = df.groupby('region_from_call2')['computedAmount'].sum().idxmax()
top_compartment = df.groupby('compartment_name_clean')['computedAmount'].sum().idxmax()

# Display executive summary
print("="*80)
print(" "*25 + "EXECUTIVE SUMMARY")
print("="*80)
print(f"\n💰 Total Cost:                  ${total_cost:,.2f}")
print(f"📅 Daily Average Cost:          ${daily_avg_cost:,.2f}")
print(f"📊 Monthly Projected Cost:      ${daily_avg_cost * 30:,.2f}")
print(f"\n🏆 Top Service:                 {top_service}")
print(f"   Cost: ${top_service_cost:,.2f} ({top_service_cost/total_cost*100:.1f}%)")
print(f"\n🌍 Top Region:                  {top_region}")
print(f"📦 Top Compartment:             {top_compartment}")
print(f"\n🔢 Total Resources:             {df['resourceId'].nunique():,}")
print(f"🔢 Active Compartments:         {df['compartment_name_clean'].nunique():,}")
print("="*80)

## 4. Cost Analysis by Service

In [ ]:
# Top 10 services by cost
service_costs = df.groupby('service')['computedAmount'].sum().sort_values(ascending=False).head(10)

print("\n📊 Top 10 Services by Cost:")
print("="*80)
for idx, (service, cost) in enumerate(service_costs.items(), 1):
    pct = (cost/total_cost)*100
    print(f"{idx:2d}. {service:40s} ${cost:>12,.2f} ({pct:>5.1f}%)")

# Visualization
plt.figure(figsize=(14, 6))
service_costs.plot(kind='barh', color='steelblue')
plt.xlabel('Cost (USD)', fontsize=12)
plt.ylabel('Service', fontsize=12)
plt.title('Top 10 Services by Cost', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
for i, v in enumerate(service_costs.values):
    plt.text(v, i, f' ${v:,.0f}', va='center', fontsize=10)
plt.tight_layout()
plt.show()

## 5. Cost Analysis by Region

In [ ]:
# Regional cost analysis
region_costs = df.groupby('region_from_call2')['computedAmount'].sum().sort_values(ascending=False)

print("\n🌍 Cost Distribution by Region:")
print("="*80)
for idx, (region, cost) in enumerate(region_costs.items(), 1):
    pct = (cost/total_cost)*100
    print(f"{idx:2d}. {region:30s} ${cost:>12,.2f} ({pct:>5.1f}%)")

# Visualization - Pie chart for top regions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Pie chart
top_regions = region_costs.head(8)
other_cost = region_costs[8:].sum()
if other_cost > 0:
    plot_data = pd.concat([top_regions, pd.Series({'Others': other_cost})])
else:
    plot_data = top_regions

ax1.pie(plot_data.values, labels=plot_data.index, autopct='%1.1f%%', startangle=90)
ax1.set_title('Cost Distribution by Region', fontsize=14, fontweight='bold')

# Bar chart
region_costs.head(10).plot(kind='barh', ax=ax2, color='coral')
ax2.set_xlabel('Cost (USD)', fontsize=12)
ax2.set_ylabel('Region', fontsize=12)
ax2.set_title('Top 10 Regions by Cost', fontsize=14, fontweight='bold')
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

## 6. Cost Analysis by Compartment

In [ ]:
# Top 15 compartments by cost
compartment_costs = df.groupby('compartment_name_clean')['computedAmount'].sum().sort_values(ascending=False).head(15)

print("\n📦 Top 15 Compartments by Cost:")
print("="*80)
for idx, (comp, cost) in enumerate(compartment_costs.items(), 1):
    pct = (cost/total_cost)*100
    print(f"{idx:2d}. {comp:40s} ${cost:>12,.2f} ({pct:>5.1f}%)")

# Visualization
plt.figure(figsize=(14, 8))
compartment_costs.plot(kind='barh', color='teal')
plt.xlabel('Cost (USD)', fontsize=12)
plt.ylabel('Compartment', fontsize=12)
plt.title('Top 15 Compartments by Cost', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
for i, v in enumerate(compartment_costs.values):
    plt.text(v, i, f' ${v:,.0f}', va='center', fontsize=10)
plt.tight_layout()
plt.show()

## 7. Time Series Analysis - Daily Cost Trends

In [ ]:
# Use df_trends (output.csv) for more granular time series analysis
# Daily cost aggregation from trends data
daily_costs = df_trends.groupby('date')['computedAmount'].sum().reset_index()
daily_costs['date'] = pd.to_datetime(daily_costs['date'])
daily_costs = daily_costs.sort_values('date')

# Calculate moving averages
daily_costs['MA3'] = daily_costs['computedAmount'].rolling(window=3, min_periods=1).mean()
daily_costs['MA7'] = daily_costs['computedAmount'].rolling(window=7, min_periods=1).mean()

print("\n📈 Daily Cost Statistics (from output.csv):")
print("="*80)
print(f"Average Daily Cost:    ${daily_costs['computedAmount'].mean():,.2f}")
print(f"Median Daily Cost:     ${daily_costs['computedAmount'].median():,.2f}")
print(f"Min Daily Cost:        ${daily_costs['computedAmount'].min():,.2f}")
print(f"Max Daily Cost:        ${daily_costs['computedAmount'].max():,.2f}")
print(f"Std Deviation:         ${daily_costs['computedAmount'].std():,.2f}")
print(f"Total Days:            {len(daily_costs)}")

# Visualization
plt.figure(figsize=(16, 6))
plt.plot(daily_costs['date'], daily_costs['computedAmount'], marker='o', label='Daily Cost', alpha=0.6, markersize=4)
plt.plot(daily_costs['date'], daily_costs['MA3'], label='3-Day Moving Avg', linewidth=2)
plt.plot(daily_costs['date'], daily_costs['MA7'], label='7-Day Moving Avg', linewidth=2)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cost (USD)', fontsize=12)
plt.title('Daily Cost Trend with Moving Averages (output.csv)', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 8. Service Cost Distribution Over Time

In [ ]:
# Use df_trends (output.csv) for service cost trends over time
# Top 5 services cost over time
top5_services = df_trends.groupby('service')['computedAmount'].sum().nlargest(5).index
service_daily = df_trends[df_trends['service'].isin(top5_services)].groupby(['date', 'service'])['computedAmount'].sum().reset_index()
service_daily['date'] = pd.to_datetime(service_daily['date'])

print("\n📊 Top 5 Services for Time Series Analysis:")
for idx, service in enumerate(top5_services, 1):
    total_cost = df_trends[df_trends['service'] == service]['computedAmount'].sum()
    print(f"{idx}. {service:30s} ${total_cost:>12,.2f}")

plt.figure(figsize=(16, 6))
for service in top5_services:
    service_data = service_daily[service_daily['service'] == service].sort_values('date')
    plt.plot(service_data['date'], service_data['computedAmount'], 
             marker='o', label=service, linewidth=2, alpha=0.7, markersize=4)

plt.xlabel('Date', fontsize=12)
plt.ylabel('Cost (USD)', fontsize=12)
plt.title('Top 5 Services - Daily Cost Trends (output.csv)', fontsize=14, fontweight='bold')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 9. Compute Instance Analysis

In [ ]:
# Filter compute resources
compute_df = df[df['service'] == 'Compute'].copy()

if len(compute_df) > 0:
    # Use skuName for shape analysis (more detailed than shape_from_call2)
    # Fill missing skuName with shape_from_call2 as fallback
    compute_df['shape_name'] = compute_df['skuName'].fillna(compute_df['shape_from_call2'])
    
    shape_costs = compute_df.groupby('shape_name')['computedAmount'].sum().sort_values(ascending=False).head(10)
    
    print("\n💻 Compute Instance Analysis:")
    print("="*80)
    print(f"Total Compute Cost:     ${compute_df['computedAmount'].sum():,.2f}")
    print(f"Unique Instances:       {compute_df['resourceId'].nunique():,}")
    print(f"Unique SKUs/Shapes:     {compute_df['shape_name'].nunique():,}")
    
    # Shape category analysis (extract shape family from skuName or shape)
    compute_df['shape_family'] = compute_df['shape_name'].str.extract(r'^(VM\.|BM\.)?([A-Za-z0-9]+)')[1]
    shape_family_costs = compute_df.groupby('shape_family')['computedAmount'].sum().sort_values(ascending=False)
    
    print("\n🔢 Top 10 SKUs/Shapes by Cost:")
    for idx, (shape, cost) in enumerate(shape_costs.items(), 1):
        instances = compute_df[compute_df['shape_name'] == shape]['resourceId'].nunique()
        avg_cost = cost / instances if instances > 0 else 0
        print(f"{idx:2d}. {shape:50s} ${cost:>12,.2f} ({instances:>3} instances, ${avg_cost:>8,.2f}/instance)")
    
    print("\n📊 Cost by Shape Family:")
    for idx, (family, cost) in enumerate(shape_family_costs.head(10).items(), 1):
        pct = (cost/compute_df['computedAmount'].sum())*100
        instances = compute_df[compute_df['shape_family'] == family]['resourceId'].nunique()
        print(f"{idx:2d}. {family:20s} ${cost:>12,.2f} ({pct:>5.1f}%) - {instances} instances")
    
    # Shape distribution by region
    print("\n🌍 Top SKUs/Shapes by Region:")
    shape_region = compute_df.groupby(['region_from_call2', 'shape_name'])['computedAmount'].sum().reset_index()
    shape_region = shape_region.sort_values('computedAmount', ascending=False).head(10)
    for idx, row in shape_region.iterrows():
        print(f"   {row['region_from_call2']:20s} | {row['shape_name']:50s} | ${row['computedAmount']:>10,.2f}")
    
    # Visualization - Multiple charts
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 12))
    
    # 1. Top 10 shapes by cost
    shape_costs.plot(kind='barh', ax=ax1, color='darkgreen')
    ax1.set_xlabel('Cost (USD)', fontsize=11)
    ax1.set_ylabel('SKU/Shape', fontsize=11)
    ax1.set_title('Top 10 Compute SKUs/Shapes by Cost', fontsize=12, fontweight='bold')
    ax1.invert_yaxis()
    for i, v in enumerate(shape_costs.values):
        ax1.text(v, i, f' ${v:,.0f}', va='center', fontsize=9)
    
    # 2. Shape family distribution (pie chart)
    top_families = shape_family_costs.head(8)
    other_cost = shape_family_costs[8:].sum()
    if other_cost > 0:
        plot_data = pd.concat([top_families, pd.Series({'Others': other_cost})])
    else:
        plot_data = top_families
    ax2.pie(plot_data.values, labels=plot_data.index, autopct='%1.1f%%', startangle=90)
    ax2.set_title('Cost Distribution by Shape Family', fontsize=12, fontweight='bold')
    
    # 3. Instance count by shape
    shape_counts = compute_df.groupby('shape_name')['resourceId'].nunique().sort_values(ascending=False).head(10)
    shape_counts.plot(kind='barh', ax=ax3, color='steelblue')
    ax3.set_xlabel('Number of Instances', fontsize=11)
    ax3.set_ylabel('SKU/Shape', fontsize=11)
    ax3.set_title('Top 10 SKUs/Shapes by Instance Count', fontsize=12, fontweight='bold')
    ax3.invert_yaxis()
    for i, v in enumerate(shape_counts.values):
        ax3.text(v, i, f' {v}', va='center', fontsize=9)
    
    # 4. Average cost per instance by shape
    shape_avg_cost = compute_df.groupby('shape_name').agg({
        'computedAmount': 'sum',
        'resourceId': 'nunique'
    })
    shape_avg_cost['avg_cost'] = shape_avg_cost['computedAmount'] / shape_avg_cost['resourceId']
    shape_avg_cost = shape_avg_cost.sort_values('avg_cost', ascending=False).head(10)
    
    shape_avg_cost['avg_cost'].plot(kind='barh', ax=ax4, color='coral')
    ax4.set_xlabel('Average Cost per Instance (USD)', fontsize=11)
    ax4.set_ylabel('SKU/Shape', fontsize=11)
    ax4.set_title('Top 10 Most Expensive SKUs/Shapes (Avg per Instance)', fontsize=12, fontweight='bold')
    ax4.invert_yaxis()
    for i, v in enumerate(shape_avg_cost['avg_cost'].values):
        ax4.text(v, i, f' ${v:,.0f}', va='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    # Shape efficiency analysis
    print("\n⚡ SKU/Shape Efficiency Analysis:")
    print("="*80)
    shape_efficiency = compute_df.groupby('shape_name').agg({
        'computedAmount': 'sum',
        'computedQuantity': 'sum',
        'resourceId': 'nunique'
    }).reset_index()
    shape_efficiency['cost_per_unit'] = shape_efficiency['computedAmount'] / shape_efficiency['computedQuantity'].replace(0, 1)
    shape_efficiency = shape_efficiency.sort_values('computedAmount', ascending=False).head(10)
    
    print("\nTop 10 SKUs/Shapes - Cost vs Usage Metrics:")
    print(f"{'SKU/Shape':<50} {'Total Cost':>12} {'Total Hours':>12} {'$/Hour':>10} {'Instances':>10}")
    print("-"*100)
    for _, row in shape_efficiency.iterrows():
        print(f"{row['shape_name']:<50} ${row['computedAmount']:>11,.2f} {row['computedQuantity']:>12,.1f} ${row['cost_per_unit']:>9,.2f} {row['resourceId']:>10}")
    
else:
    print("\n⚠️ No compute resources found in the dataset")

## 9a. Top Resource Consumers Analysis

In [ ]:
# Top 20 resource consumers by cost
resource_costs = df.groupby(['resourceName', 'service', 'region_from_call2']).agg({
    'computedAmount': 'sum',
    'computedQuantity': 'sum',
    'resourceId': 'first',
    'compartment_name_clean': 'first'
}).reset_index()
resource_costs = resource_costs.sort_values('computedAmount', ascending=False).head(20)

print("\n🏆 Top 20 Resource Consumers by Cost:")
print("="*120)
print(f"{'Rank':<5} {'Resource Name':<35} {'Service':<20} {'Region':<15} {'Cost':>12} {'Compartment':<25}")
print("-"*120)

for idx, row in resource_costs.iterrows():
    rank = resource_costs.index.get_loc(idx) + 1
    resource_name = str(row['resourceName'])[:33] if pd.notna(row['resourceName']) else 'N/A'
    service = str(row['service'])[:18]
    region = str(row['region_from_call2'])[:13]
    cost = row['computedAmount']
    compartment = str(row['compartment_name_clean'])[:23]
    
    print(f"{rank:<5} {resource_name:<35} {service:<20} {region:<15} ${cost:>11,.2f} {compartment:<25}")

# Calculate statistics
total_top20_cost = resource_costs['computedAmount'].sum()
pct_of_total = (total_top20_cost / df['computedAmount'].sum()) * 100

print("\n📊 Summary:")
print(f"Total cost of Top 20 resources:     ${total_top20_cost:,.2f}")
print(f"Percentage of total cost:           {pct_of_total:.1f}%")
print(f"Average cost per resource (Top 20): ${resource_costs['computedAmount'].mean():,.2f}")

# Visualization - Top 20 resources
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# Bar chart of top 20
resource_names_short = resource_costs['resourceName'].apply(lambda x: str(x)[:30] + '...' if pd.notna(x) and len(str(x)) > 30 else str(x))
ax1.barh(range(len(resource_costs)), resource_costs['computedAmount'], color='darkblue')
ax1.set_yticks(range(len(resource_costs)))
ax1.set_yticklabels(resource_names_short, fontsize=9)
ax1.set_xlabel('Cost (USD)', fontsize=12)
ax1.set_ylabel('Resource Name', fontsize=12)
ax1.set_title('Top 20 Resource Consumers by Cost', fontsize=13, fontweight='bold')
ax1.invert_yaxis()
for i, v in enumerate(resource_costs['computedAmount'].values):
    ax1.text(v, i, f' ${v:,.0f}', va='center', fontsize=8)

# Service distribution for top 20
service_dist = resource_costs.groupby('service')['computedAmount'].sum().sort_values(ascending=False)
colors = plt.cm.Set3(range(len(service_dist)))
ax2.pie(service_dist.values, labels=service_dist.index, autopct='%1.1f%%', startangle=90, colors=colors)
ax2.set_title('Service Distribution of Top 20 Resources', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

# Additional breakdown by service
print("\n📦 Top 20 Resources - Breakdown by Service:")
service_breakdown = resource_costs.groupby('service').agg({
    'computedAmount': 'sum',
    'resourceName': 'count'
}).sort_values('computedAmount', ascending=False)
service_breakdown.columns = ['Total_Cost', 'Resource_Count']

for service, row in service_breakdown.iterrows():
    print(f"  {service:25s} {row['Resource_Count']:>2} resources | ${row['Total_Cost']:>12,.2f}")

# Regional distribution
print("\n🌍 Top 20 Resources - Regional Distribution:")
region_breakdown = resource_costs.groupby('region_from_call2').agg({
    'computedAmount': 'sum',
    'resourceName': 'count'
}).sort_values('computedAmount', ascending=False)
region_breakdown.columns = ['Total_Cost', 'Resource_Count']

for region, row in region_breakdown.iterrows():
    print(f"  {region:25s} {row['Resource_Count']:>2} resources | ${row['Total_Cost']:>12,.2f}")

## 10. Storage Analysis

In [ ]:
# Filter storage services
storage_services = ['Block Storage', 'Object Storage', 'File Storage', 'Archive Storage']
storage_df = df[df['service'].isin(storage_services)].copy()

if len(storage_df) > 0:
    storage_costs = storage_df.groupby('service')['computedAmount'].sum().sort_values(ascending=False)
    
    print("\n💾 Storage Analysis:")
    print("="*120)
    print(f"Total Storage Cost:     ${storage_df['computedAmount'].sum():,.2f}")
    print(f"Total Storage Usage:    {storage_df['computedQuantity'].sum():,.2f} units")
    
    print("\n📊 Storage Costs by Type:")
    for idx, (service, cost) in enumerate(storage_costs.items(), 1):
        pct = (cost/storage_df['computedAmount'].sum())*100
        usage = storage_df[storage_df['service'] == service]['computedQuantity'].sum()
        print(f"{idx}. {service:30s} ${cost:>12,.2f} ({pct:>5.1f}%) | Usage: {usage:>12,.2f}")
    
    # Storage by compartment with full path
    print("\n📦 Storage Costs by Compartment (with Full Path):")
    storage_compartment = storage_df.groupby(['compartment_name_clean', 'compartmentPath']).agg({
        'computedAmount': 'sum',
        'computedQuantity': 'sum',
        'resourceId': 'nunique'
    }).reset_index().sort_values('computedAmount', ascending=False).head(15)
    
    print(f"{'Rank':<5} {'Compartment':<30} {'Cost':>12} {'Usage':>12} {'Resources':>10}")
    print(f"{'':5} {'Full Path':<100}")
    print("-"*120)
    for idx, row in storage_compartment.iterrows():
        rank = idx + 1
        pct = (row['computedAmount']/storage_df['computedAmount'].sum())*100
        print(f"{rank:<5} {row['compartment_name_clean'][:28]:<30} ${row['computedAmount']:>11,.2f} {row['computedQuantity']:>12,.1f} {row['resourceId']:>10} ({pct:>4.1f}%)")
        print(f"{'':5} {row['compartmentPath'][:98]}")
        print()
    
    # Storage by service and compartment (top combinations) with full path
    print("\n🔍 Top 10 Storage Service-Compartment Combinations:")
    service_comp = storage_df.groupby(['service', 'compartment_name_clean', 'compartmentPath']).agg({
        'computedAmount': 'sum'
    }).reset_index().sort_values('computedAmount', ascending=False).head(10)
    
    print(f"{'Rank':<5} {'Service':<20} {'Compartment':<30} {'Cost':>12}")
    print(f"{'':5} {'Full Path':<100}")
    print("-"*120)
    for idx, row in service_comp.iterrows():
        rank = idx + 1
        print(f"{rank:<5} {row['service']:<20} {row['compartment_name_clean'][:28]:<30} ${row['computedAmount']:>11,.2f}")
        print(f"{'':5} {row['compartmentPath'][:98]}")
        print()
    
    # Visualization - Multiple charts
    fig = plt.figure(figsize=(18, 12))
    gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)
    
    # 1. Pie chart - Storage type distribution
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.pie(storage_costs.values, labels=storage_costs.index, autopct='%1.1f%%', startangle=90)
    ax1.set_title('Storage Cost Distribution by Type', fontsize=13, fontweight='bold')
    
    # 2. Bar chart - Storage by region
    ax2 = fig.add_subplot(gs[0, 1])
    storage_region = storage_df.groupby('region_from_call2')['computedAmount'].sum().sort_values(ascending=False).head(10)
    storage_region.plot(kind='barh', ax=ax2, color='orange')
    ax2.set_xlabel('Cost (USD)', fontsize=11)
    ax2.set_ylabel('Region', fontsize=11)
    ax2.set_title('Storage Costs by Region', fontsize=13, fontweight='bold')
    ax2.invert_yaxis()
    for i, v in enumerate(storage_region.values):
        ax2.text(v, i, f' ${v:,.0f}', va='center', fontsize=9)
    
    # 3. Bar chart - Top 15 compartments
    ax3 = fig.add_subplot(gs[1, :])
    compartment_costs = storage_compartment.set_index('compartment_name_clean')['computedAmount']
    ax3.barh(range(len(compartment_costs)), compartment_costs.values, color='teal')
    ax3.set_yticks(range(len(compartment_costs)))
    ax3.set_yticklabels([str(x)[:38] for x in compartment_costs.index], fontsize=9)
    ax3.set_xlabel('Cost (USD)', fontsize=11)
    ax3.set_ylabel('Compartment', fontsize=11)
    ax3.set_title('Top 15 Compartments by Storage Cost', fontsize=13, fontweight='bold')
    ax3.invert_yaxis()
    for i, v in enumerate(compartment_costs.values):
        ax3.text(v, i, f' ${v:,.0f}', va='center', fontsize=9)
    
    # 4. Stacked bar - Storage types by compartment
    ax4 = fig.add_subplot(gs[2, :])
    top_comps = storage_compartment.head(10)['compartment_name_clean'].values
    storage_pivot = storage_df[storage_df['compartment_name_clean'].isin(top_comps)].pivot_table(
        index='compartment_name_clean', 
        columns='service', 
        values='computedAmount', 
        aggfunc='sum', 
        fill_value=0
    )
    storage_pivot = storage_pivot.reindex(top_comps)  # Maintain order
    storage_pivot.plot(kind='barh', stacked=True, ax=ax4, colormap='Set3')
    ax4.set_xlabel('Cost (USD)', fontsize=11)
    ax4.set_ylabel('Compartment', fontsize=11)
    ax4.set_title('Storage Type Distribution by Top 10 Compartments', fontsize=13, fontweight='bold')
    ax4.legend(title='Storage Type', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
    ax4.invert_yaxis()
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics by compartment
    print("\n📈 Storage Summary by Compartment:")
    print(f"Total Compartments with Storage:  {storage_df['compartment_name_clean'].nunique()}")
    print(f"Average Cost per Compartment:     ${storage_df.groupby('compartment_name_clean')['computedAmount'].sum().mean():,.2f}")
    print(f"Top Compartment Cost:             ${storage_compartment['computedAmount'].max():,.2f}")
    print(f"Top Compartment Usage:            {storage_compartment['computedQuantity'].max():,.2f} units")
    
else:
    print("\n⚠️ No storage resources found in the dataset")

## 11. Cost Anomaly Detection

In [ ]:
# Detect anomalies using statistical methods
daily_costs_sorted = daily_costs.sort_values('date').reset_index(drop=True)
mean_cost = daily_costs_sorted['computedAmount'].mean()
std_cost = daily_costs_sorted['computedAmount'].std()

# Define anomalies as values beyond 2 standard deviations
daily_costs_sorted['is_anomaly'] = np.abs(daily_costs_sorted['computedAmount'] - mean_cost) > (2 * std_cost)
anomalies = daily_costs_sorted[daily_costs_sorted['is_anomaly']]

print("\n⚠️ Cost Anomaly Detection:")
print("="*80)
print(f"Threshold: ±2 Standard Deviations from Mean")
print(f"Mean Daily Cost:      ${mean_cost:,.2f}")
print(f"Standard Deviation:   ${std_cost:,.2f}")
print(f"Upper Threshold:      ${mean_cost + 2*std_cost:,.2f}")
print(f"Lower Threshold:      ${mean_cost - 2*std_cost:,.2f}")
print(f"\nAnomalies Detected:   {len(anomalies)}")

if len(anomalies) > 0:
    print("\n📅 Anomalous Days:")
    for _, row in anomalies.iterrows():
        deviation = ((row['computedAmount'] - mean_cost) / mean_cost) * 100
        print(f"  {row['date']}: ${row['computedAmount']:,.2f} ({deviation:+.1f}% from mean)")
    
    # Visualization
    plt.figure(figsize=(16, 6))
    plt.plot(daily_costs_sorted['date'], daily_costs_sorted['computedAmount'], 
             marker='o', label='Daily Cost', color='blue', alpha=0.6)
    plt.axhline(y=mean_cost, color='green', linestyle='--', label='Mean', linewidth=2)
    plt.axhline(y=mean_cost + 2*std_cost, color='red', linestyle='--', label='Upper Threshold', linewidth=2)
    plt.axhline(y=mean_cost - 2*std_cost, color='red', linestyle='--', label='Lower Threshold', linewidth=2)
    
    # Highlight anomalies
    if len(anomalies) > 0:
        plt.scatter(anomalies['date'], anomalies['computedAmount'], 
                   color='red', s=200, zorder=5, label='Anomalies', marker='X')
    
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Cost (USD)', fontsize=12)
    plt.title('Cost Anomaly Detection (±2σ)', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("\n✅ No anomalies detected in the period")

## 12. Cost Optimization Opportunities

In [ ]:
print("\n💡 Cost Optimization Opportunities:")
print("="*80)

# 1. Idle resources (very low usage)
resource_usage = df.groupby('resourceId').agg({
    'computedAmount': 'sum',
    'computedQuantity': 'sum',
    'service': 'first'
}).reset_index()

# Resources with cost but minimal usage
potential_idle = resource_usage[(resource_usage['computedAmount'] > 0) & 
                                (resource_usage['computedQuantity'] < 1)]

print(f"\n1️⃣ Potentially Idle Resources: {len(potential_idle)}")
if len(potential_idle) > 0:
    idle_cost = potential_idle['computedAmount'].sum()
    print(f"   Potential Savings: ${idle_cost:,.2f}")
    print("\n   Top 5 by Cost:")
    for idx, row in potential_idle.nlargest(5, 'computedAmount').iterrows():
        print(f"   - {row['service']:20s} ${row['computedAmount']:>10,.2f}")

# 2. Multi-region resources that could be consolidated
multi_region_services = df.groupby('service')['region_from_call2'].nunique()
multi_region_services = multi_region_services[multi_region_services > 3].sort_values(ascending=False)

print(f"\n2️⃣ Services Spanning Multiple Regions (3+):")
for service, count in multi_region_services.head(5).items():
    service_cost = df[df['service'] == service]['computedAmount'].sum()
    print(f"   - {service:30s} {count} regions, ${service_cost:,.2f}")

# 3. High-cost compartments that need review
high_cost_compartments = df.groupby('compartment_name_clean')['computedAmount'].sum().nlargest(5)
print(f"\n3️⃣ Top 5 High-Cost Compartments (Review for optimization):")
for comp, cost in high_cost_compartments.items():
    pct = (cost/total_cost)*100
    print(f"   - {comp:40s} ${cost:>10,.2f} ({pct:.1f}%)")

print("\n💡 Recommendation: Review Cloud Advisor recommendations for detailed optimization guidance")
print("   Run: ./collector.sh <params> --only-recommendations")

## 13. Cost Forecast (Simple Linear Projection)

In [ ]:
# Simple linear forecast for next 7 days
from sklearn.linear_model import LinearRegression

# Prepare data
daily_costs_forecast = daily_costs_sorted.copy()
daily_costs_forecast['day_num'] = range(len(daily_costs_forecast))

X = daily_costs_forecast[['day_num']].values
y = daily_costs_forecast['computedAmount'].values

# Train model
model = LinearRegression()
model.fit(X, y)

# Forecast next 7 days
future_days = np.array(range(len(daily_costs_forecast), len(daily_costs_forecast) + 7)).reshape(-1, 1)
forecast = model.predict(future_days)

last_date = daily_costs_forecast['date'].max()
forecast_dates = pd.date_range(start=last_date + timedelta(days=1), periods=7)

print("\n📊 7-Day Cost Forecast (Linear Projection):")
print("="*80)
total_forecast = 0
for date, cost in zip(forecast_dates, forecast):
    print(f"{date.strftime('%Y-%m-%d')}: ${cost:,.2f}")
    total_forecast += cost
print(f"\nForecast Total (7 days): ${total_forecast:,.2f}")
print(f"Forecast Monthly:        ${total_forecast * 30 / 7:,.2f}")

# Visualization
plt.figure(figsize=(16, 6))
plt.plot(daily_costs_forecast['date'], daily_costs_forecast['computedAmount'], 
         marker='o', label='Historical', color='blue', linewidth=2)
plt.plot(forecast_dates, forecast, marker='s', label='Forecast', 
         color='red', linestyle='--', linewidth=2)
plt.axvline(x=last_date, color='gray', linestyle=':', linewidth=2, label='Today')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cost (USD)', fontsize=12)
plt.title('7-Day Cost Forecast', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 14. Export Key Insights to CSV

In [ ]:
# Export top spenders by different dimensions
print("\n📁 Exporting Key Insights:")
print("="*80)

# Top services
top_services_df = df.groupby('service')['computedAmount'].sum().reset_index()
top_services_df.columns = ['Service', 'Total_Cost']
top_services_df = top_services_df.sort_values('Total_Cost', ascending=False)
top_services_df.to_csv('../output/analysis_top_services.csv', index=False)
print(f"✅ Exported: ../output/analysis_top_services.csv ({len(top_services_df)} services)")

# Top compartments
top_compartments_df = df.groupby(['compartment_name_clean', 'compartmentPath'])['computedAmount'].sum().reset_index()
top_compartments_df.columns = ['Compartment', 'Full_Path', 'Total_Cost']
top_compartments_df = top_compartments_df.sort_values('Total_Cost', ascending=False)
top_compartments_df.to_csv('../output/analysis_top_compartments.csv', index=False)
print(f"✅ Exported: ../output/analysis_top_compartments.csv ({len(top_compartments_df)} compartments)")

# Daily costs
daily_costs_export = daily_costs_sorted[['date', 'computedAmount', 'MA3', 'MA7']]
daily_costs_export.columns = ['Date', 'Daily_Cost', 'MA_3Day', 'MA_7Day']
daily_costs_export.to_csv('../output/analysis_daily_costs.csv', index=False)
print(f"✅ Exported: ../output/analysis_daily_costs.csv ({len(daily_costs_export)} days)")

# Resource-level analysis
resource_summary = df.groupby(['resourceId', 'service', 'region_from_call2']).agg({
    'computedAmount': 'sum',
    'computedQuantity': 'sum',
    'compartment_name_clean': 'first'
}).reset_index()
resource_summary.columns = ['Resource_ID', 'Service', 'Region', 'Total_Cost', 'Total_Quantity', 'Compartment']
resource_summary = resource_summary.sort_values('Total_Cost', ascending=False)
resource_summary.to_csv('../output/analysis_resource_summary.csv', index=False)
print(f"✅ Exported: ../output/analysis_resource_summary.csv ({len(resource_summary)} resources)")

print("\n✅ All insights exported successfully!")

## 15. Custom Query Section

Use this section to run custom queries on the data:

In [ ]:
# Example: Filter by specific service
# service_name = 'Compute'
# service_data = df[df['service'] == service_name]
# print(f"Total {service_name} cost: ${service_data['computedAmount'].sum():,.2f}")

# Example: Filter by date range
# start_date = '2025-11-01'
# end_date = '2025-11-10'
# date_filtered = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
# print(f"Cost in date range: ${date_filtered['computedAmount'].sum():,.2f}")

# Add your custom queries here:

## Summary

This notebook provides comprehensive FinOps intelligence including:
- ✅ Executive summary with key metrics
- ✅ Cost breakdowns by service, region, and compartment
- ✅ Time series analysis and trends
- ✅ Compute and storage deep dives
- ✅ Anomaly detection
- ✅ Cost optimization opportunities
- ✅ 7-day cost forecast
- ✅ Exportable insights for reporting

### Next Steps:
1. Review anomalies and investigate root causes
2. Analyze high-cost compartments and services
3. Check Cloud Advisor recommendations: `./collector.sh <params> --only-recommendations`
4. Implement cost optimization strategies
5. Set up regular monitoring and reporting